In [ ]:
### Transform random forest classification results (or other .tif files in a folder) to shapefiles. Faster than the respective function in R. ### 
### adapted from: http://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html#polygonize-a-raster-band ###

from osgeo import gdal, ogr, osr
import os, fnmatch

# Path to folder containing .tif raster files. The result is stored in the same location.
in_path = ''

files = os.listdir(in_path)
in_rasters = []
pattern = "*.tif"  
for entry in files:  
    if fnmatch.fnmatch(entry, pattern):
            filename = in_path + '/' + entry
            in_rasters.append(filename)

#  get raster datasource
for raster in in_rasters:
    my_raster = gdal.Open(raster)
    my_band = my_raster.GetRasterBand(1)
    proj = my_raster.GetProjection() 
    src = osr.SpatialReference()
    src.ImportFromWkt(proj)
    
    #  create output datasource
    my_layername = raster.split('.tif')[0]
    drv = ogr.GetDriverByName('ESRI Shapefile')
    dst_ds = drv.CreateDataSource(my_layername + '.shp')
    dst_layer = dst_ds.CreateLayer(my_layername, srs = src)
    
    newField = ogr.FieldDefn('id', ogr.OFTInteger)
    dst_layer.CreateField(newField)
            
    gdal.Polygonize(my_band, None, dst_layer, 0, [], callback = None)
    
    dst_ds.Destroy()
    my_raster = None
    my_band = None